# Training environment information

In [1]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        716M          9G        1.2M        2.0G         11G
Swap:            0B          0B          0B


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-913d5027-9e38-451d-25e5-30c7519f130c)


In [3]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [4]:
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  1


In [5]:
!nvidia-smi

Mon Jun 20 15:17:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [7]:
!pip install fastai==2.5.3 -q
!pip freeze

     |████████████████████████████████| 189 kB 4.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x39e1a000 @  0x7f08e9bcc615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 23.2 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.10.2 which is incompatible.
torchaudio 0.11.0+cu113 requires torc

In [8]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [9]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path="'/.'"
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path,copy_model_to_google_drive
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis
from eb_ml_utils import save_model_weights,build_data_loader,build_and_train_learner,score_model

# Training Configuration

In [15]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-May2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']


# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

Running on COLAB
Mounted at /gdrive


Source data: battery EIS performed for SoC 100,90,80,70,60,50,40,30,20,10. Same frequnecies for each SoC

Data format:  Re{Zbat}-Im{Zbat}j

```Matlab
prefix = {...
    './Batt1/Meas1/'; --> 1
    './Batt1/Meas2/'; --> 2*
	  './Batt2/Meas1/'; --> 3  
    './Batt2/Meas2/'; --> 4*
    './Batt3/Meas1/'; --> 5
    './Batt3/Meas2/'; --> 6* ...
    './Batt4/Meas1/';
    './Batt5/Meas1/';
    './Batt6/Meas1/';
    './Batt7/Meas1/';
    './Batt8/Meas1/';
    './Batt9/Meas1/';
    './Batt10/Meas1/'; -->13
    };
```

# Data Selection

In [13]:

# Data acquition file to load from dateset folder
all_batteries=["02_4","02_5","02_6","02_7","02_8","02_9","03_4","03_5","03_6","03_7","03_8","03_9","05_3","05_4","05_5","05_6","05_7","05_8"] # batteria 6 usate per test

# List of data acquisition to use for cross validation
# A cross- validation experiment will be generate for each item in the list
cross_validation_list=[["02_4","02_5","02_6","02_7","02_8","02_9"],["03_4","03_5","03_6","03_7","03_8","03_9"],["05_5","05_6","05_7","05_8"]] # Data acquitions on battery 6 used for testing
cross_validation_experiment_names = ["02","03","05"]

# Model Training

## Leave One Battery Out

In [16]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_leave_one_out_"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'

model_accuracy={}
cross_validation_experiment_index = 0 # Cross validation experiment index

# Train and score model for cross-validation
for batt_measures in cross_validation_list:
  batt=cross_validation_experiment_names[cross_validation_experiment_index]
  battery_list=all_batteries.copy()
  test_battery_list = []
  for batt_mes in batt_measures:
    battery_list.remove(batt_mes)
    test_battery_list.append(batt_mes)
  print("battery data acquisitions for training and validation: ")
  print(battery_list)
  print("battery data acquisitions for test: ")
  print(test_battery_list)
  model_name=experiment_name_prefix+str(batt)
  print("model name: "+model_name)

  config['ExperimentName'] = model_name
  config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
  config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

  #GENERATE IMAGE
  if(generate_images):
      dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
      generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

      test_dataset,feature_col_names=load_soc_dataset(test_battery_list,config["soc_list"],config['DATASETS_DIR'])
      generate_image_files_from_eis(test_dataset,feature_col_names,config['TEST_IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

  # TRAINING
  learn= build_and_train_learner(config,n_epochs=n_epochs)
  #SAVE
  weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
  filename_pth= weights_filename+".pth"
  copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
  # SCORE MODEL
  dl=build_data_loader(config)
  model_accuracy[model_name]=score_model(weights_filename,dl)
  cross_validation_experiment_index+=1

battery data acquisitions for training and validation: 
['03_4', '03_5', '03_6', '03_7', '03_8', '03_9', '05_3', '05_4', '05_5', '05_6', '05_7', '05_8']
battery data acquisitions for test: 
['02_4', '02_5', '02_6', '02_7', '02_8', '02_9']
model name: Paper_EIS_leave_one_out_02
dataset row number: 120
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_02
df_real shape: (120, 14)
df_img shape: (120, 14)
soc: 100
battery: 03_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_02/Paper_EIS_leave_one_out_02/EIS_BATT03_4_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_02/Paper_EIS_leave_one_out_02/EIS_BATT1100_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_02/Paper_EIS_leave_one_out_02/EIS_BATT1200_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_02/Paper_EIS_leave_one_out_02/EIS_BATT1300_100.png
augmentation_index: 4
/gdrive/MyDrive/batteri

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Valley: 6.92e-04


epoch,train_loss,valid_loss,accuracy,time
0,3.351527,2.402497,0.198718,00:06


...

epoch,train_loss,valid_loss,accuracy,time
0,2.071050,1.006478,0.635802,00:06
1,1.884350,0.944968,0.620370,00:06
2,1.668874,0.796554,0.691358,00:06
3,1.507476,0.731717,0.706790,00:06
4,1.393036,0.585082,0.783951,00:06
5,1.259768,0.629613,0.753086,00:06
6,1.135877,0.531770,0.802469,00:06
7,1.048057,0.447115,0.854938,00:06
8,0.994641,0.563998,0.787037,00:06
9,0.913492,0.547080,0.780864,00:06


saved filename: Paper_EIS_leave_one_out_05_1655739790.227648_SAVED.pth
Setting-up type transforms pipelines
Found 1080 items
2 datasets of sizes 756,324
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_05/Paper_EIS_leave_one_out_05/EIS_BATT1878_020.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_leave_one_out_05/Paper_EIS_leave_one_out_05/EIS_BATT1878_020.png
    applying RegexLabeller gives
      020
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(1)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(1))


Found 1080 items
2 datasets of sizes 75

[[32  0  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  0  0  0  0  0  2]
 [ 0  1 25  3  0  0  0  0  0  1]
 [ 0  0 10 25  0  0  0  0  0  0]
 [ 0  0  0  1 29  0  0  0  0  0]
 [ 0  0  0  0  0 37  0  1  0  0]
 [ 0  0  0  0  0  0 24  3  0  0]
 [ 0  0  0  0  0  0  5 25  2  0]
 [ 0  0  0  0  0  0  0  3 32  0]
 [ 0  0  1  0  0  0  0  0  0 27]]
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.94594595 0.         0.         0.         0.
  0.         0.         0.         0.05405405]
 [0.         0.03333333 0.83333333 0.1        0.         0.
  0.         0.         0.         0.03333333]
 [0.         0.         0.28571429 0.71428571 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.03333333 0.96666667 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.97368421
  0.         0.02631579 0.         0.        ]
 [0.         0.    

learn.validate()


Model accuracy: 0.90
learn.get_preds()
